In [1]:
from google.colab import drive
drive.mount("/content/drive")
import csv
import re
import tqdm.notebook as tq
import pickle
import pandas as pd

Mounted at /content/drive


In [2]:
data_path = '/content/drive/Shareddrives/LOPE/產學合作/中華電信/2020-2021/指向情緒案/data/annot_data/annotated_data_bkup/20210412/aspect_tuples_20210412.1.csv'

with open(data_path, 'r', encoding='utf-8') as f:
  rows = csv.reader(f, delimiter = ',')
  headers = next(rows)
  all_data = [ row for row in rows ]
  evaltext = [ data[6] for data in all_data if data[3] == 'True'] # filter evaltext whose is_context is True

In [29]:
construction_path = '/content/drive/Shareddrives/LOPE/產學合作/中華電信/2020-2021/指向情緒案/sentiment-construction-list/data/constructions_0415.csv'

with open(construction_path) as f:
    constructions = [ row.strip() for row in f ]

In [3]:
noise_path = '/content/drive/Shareddrives/LOPE/產學合作/中華電信/2020-2021/指向情緒案/data/annot_data/annotated_data_bkup/20210412/noise_pairs_20210412.pkl'

with open(noise_path, 'rb') as f:
  noise = pickle.load(f)
  noise_text = [ data[0] for data in noise ]

In [24]:
seeds_path = '/content/drive/Shareddrives/LOPE/產學合作/中華電信/2020-2021/指向情緒案/sentiment-construction-list/data/seeds.csv'

with open(seeds_path, 'r', encoding='utf-8') as f:
  rows = csv.reader(f, delimiter = ',')
  seeds = [ ' '.join(row) for row in rows ]

seeds_pat = "(%s)" % "|".join(seeds)

In [17]:
# find matched seeds in evaltext
matched_seeds = []

for text in evaltext:
  for seed in seeds:
    if seed in text:
      matched_seeds.append(seed)

matched_seeds_pat = re.compile("(%s)" % "|".join(matched_seeds))

In [6]:
# replace seeds in evaltext with `__SLOT__` to create new constructions
all_new_cnstr = []
for text in evaltext:
  new_cnstr = re.sub(matched_seeds_pat, '__SLOT__', text)
  all_new_cnstr.append(new_cnstr)

In [25]:
# find if new constructions are found in noise_text
matched = {}
matched_count = {}
for k in tq.tqdm(all_new_cnstr):
  ori_key = k
  k = re.escape(k)
  k = k.replace('__SLOT__', seeds_pat) # replace `__SLOT__` with the seed if there is any
  try:
    pat = re.compile(k)
  except:
    print(k)
    break
  
  # Get all cnstr from all comments
  candidates = set()
  for text in noise_text:
    for c in pat.finditer(text): candidates.add(c[0])
  
  # Count all cnstr
  matched_count[ori_key] = len(candidates)

  if len(candidates) > 1:
    if ori_key not in matched: matched[ori_key] = []
    for c in candidates: matched[ori_key].append(c)

In [27]:
# show all new cnstr found in noise_text
df = pd.DataFrame([matched_count, matched]).T
df.columns = ['count', 'examples']
df = df[df['count']>1]
df = df.sort_values(by='count', ascending=False)
df

,count,examples
__SLOT__,474,"[請私訊, 地下室, 網外, 拆台, 兩格, 山郊, 無框, speedtest, 連不上,..."
都__SLOT__,18,"[都訊號, 都超爛, 都續約, 都市區, 都1399, 都吃到飽, 都沒訊號, 都有訊號, ..."
__SLOT__11,10,"[預付卡11, 儲11, 遠傳11, LM11, iphone11, 合約11, 網外11,..."
__SLOT__最快,3,"[速度最快, 88最快, 吃到飽最快]"


In [30]:
# find if original constructions are found in noise_text
matched = {}
matched_con = {}
for k in tq.tqdm(constructions):
  pat = re.compile(k)
  if k not in matched: matched[k] = []
  
  # Get all cnstr from all comments
  candidates = set()
  for cmt in noise_text:
    for c in pat.finditer(cmt): candidates.add(c[0])
  
  # Count all cnstr
  matched_con[k] = len(candidates)

  if len(candidates) > 1:
    for c in candidates: matched[k].append(c)

In [36]:
# show all original cnstr found in noise_text
df = pd.DataFrame([matched_con, matched]).T
df.columns = ['count', 'examples']
df = df[df['count'] >1]
df = df.sort_values(by='count', ascending=False)
df

,count,examples
都\w+了\b,62,"[都跟客人說去超商問店員就好了, 都好了, 都要搶開台5g了, 都跑不到10M了, 都不願意..."
不是\w+.?是\w+\b,25,"[不是最低但是已經算低了, 不是一樣是不限速吃到飽剛好我剩下的約期差不多也是12個月體驗完也..."
越\w越\w+\b,14,"[越來越期待5G帶來的速度了, 越來越少營收變多但用戶的使用品質會越來越下降, 越多越爽, ..."
也太\w+了?吧?\b,12,"[也太便宜wwwww, 也太大, 也太慢了吧, 也太好用了吧, 也太不給力了, 也太晚了吧標..."
直接\w+了\b,12,"[直接去那間辦還行吧不過不是住附近的算上交通費就不划算了, 直接降到499了, 直接辦好了,..."
坐等\w+\b,8,"[坐等台之星5G開台, 坐等5g, 坐等299, 坐等熱點無限方案, 坐等老大先開台, 坐等..."
\b\w+到一個\w+\b,7,"[還有不到一個月到期, 手上這支剛買不到一個月, 亞太還要辦到一個月超過300的方案, 剛購..."
(真的)?有夠\w+的?\b,4,"[有夠兇, 有夠貴, 有夠爛, 有夠大昨天有小豬說台廠已經出局]"
\b\w+死了,4,"[一個月1399貴死了, 網路慢死了, 一句話就能把大家都打死了, 卡到炸爛死了]"
\b\w+不少\b,4,"[k還多不少, 覺得中華這二年基地台建設不少, 的比例應該不少, 688通話送不少]"
